In [ ]:
pip install pandas openpyxl requests tqdm

In [ ]:
import pandas as pd
import requests
from tqdm import tqdm

def get_route_info(source, destination):

    base_url = "https://router.project-osrm.org/route/v1/driving/"
    source_str = f"{source[0]},{source[1]}"
    destination_str = f"{destination[0]},{destination[1]}"
    
    # Construct the full URL
    url = f"{base_url}{source_str};{destination_str}?overview=false"
    
    try:
        # Send GET request to OSRM API
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses
        
        # Parse JSON response
        data = response.json()
        
        # Extract required details
        if data.get("code") == "Ok":
            distance = data["routes"][0]["distance"] / 1000  # Convert meters to kilometers
            duration = data["routes"][0]["duration"] / 3600  # Convert seconds to hours
            return {"distance": distance, "duration": duration}
        else:
            raise ValueError(f"Error in response: {data.get('code')}")
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

def process_coordinates(input_file, output_file):

    # Read input data from Excel file
    df = pd.read_excel(input_file)
    
    # Add new columns for distance and duration
    df['distance'] = None
    df['duration'] = None
    
    # Use tqdm to create a progress bar
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
        source_coords = (row['source longitude'], row['source latitude'])
        destination_coords = (row['destination longitude'], row['destination latitude'])
        
        # Fetch distance and duration
        result = get_route_info(source_coords, destination_coords)
        
        if result:
            df.at[index, 'distance'] = result['distance']  # in kilometers
            df.at[index, 'duration'] = result['duration']  # in hours
    
    # Write the updated data to the output Excel file
    df.to_excel(output_file, index=False)

# Example usage
input_file = 'coordinates.xlsx'  # Replace with your input Excel file path
output_file = 'coordinates_with_results1.xlsx'  # Replace with your desired output file path

process_coordinates(input_file, output_file)